In [2]:
from src.inference.segment_inference import inference
from PIL import Image 


In [29]:
from src.inference.segment_inference import inference
import cv2
image_origin = cv2.imread('./test.jpg')
outputs = inference(image_origin, model_path="./src/model/segment.onnx")



In [30]:
def crop_image(image, box):

    x1, y1, x2, y2 = map(int, box)  # Convert coordinates to integers
    cropped_image = image[y1:y2, x1:x2]
    return cropped_image

In [31]:
from pytesseract import pytesseract 

path_to_tesseract = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

pytesseract.tesseract_cmd = path_to_tesseract 
def extract_text(outputs):
    for i in range(len(outputs)):
        image = crop_image(image_origin, outputs[i].get("box"))
        text = pytesseract.image_to_string(image)
        outputs[i].update({"image":image})
        outputs[i].update({"text":text})
    return outputs

In [32]:
outputs = extract_text(outputs)

In [34]:
for i in range(len(outputs)):
    cv2.imshow((outputs[i].get("label").upper()),outputs[i].get("image"))
    cv2.waitKey(0)
    cv2.destroyAllWindows()
cv2.destroyAllWindows()

In [35]:
for i in range(len(outputs)):
    print((outputs[0].get("label").upper()) + ": ", outputs[i].get("text"))
    print("=========================")

EXPERIENCE:  EXPERIENCE PROFESSIONNELLE

EXPERIENCE:  
EXPERIENCE:  OMPETEN:


EXPERIENCE:  Mme NAJI Chaimae

EXPERIENCE:  ‘Telephone : 06-59.47.56.32
E-mail: cnajighepmistes.net
Linkedin shetpsd/unnaclinkedin.com/
infehaienae-niaji-OOs9s7192
bourgegnes, 95000 CuRGY

amecaceanaes SEATTA

EXPERIENCE:  FORMATION PROFESSIONNELLE


EXPERIENCE:  DE 12 SEMAINES A PARTIR DU: 08/11/2021
GuaaurAo. weloaieasa

EXPERIENCE:  EN RECHERCHE D'UN STAGE TECHNICIEN EN
an cuene

